<img src='../images/bildungscampus_logo.png' width="40%" align="left" />
<img src='../images/hhn.png' width="25%" align="right" />

# Schritt 1: Web Scraping and Storing
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='../images/bar.png'/>

## Extraktion und Speicherung von Informationen aus Websiten

Bevor mit der eigentlichen Entwicklung der Chatbot-Kmponenten begonnen werden kann, müssen die Informationen, auf deren Basis der Chatbot arbeiten soll, beschafft und in einer Datenbank gespeichert werden. Das Wissen des Chatbots muss also beschafft werden. Im Rahmen dieser soll der Chatbot Prototyp für HHN Studierende ein Ansprechpartner sein, der Kontaktdaten und Forschungsgebiete der Professoren der HHN kennt. Darum werden diese Informationen im ersten Schritt durch Web Scraping extrahiert und in eine Datenbank gespeichert. 

### 1.1. Import der Bibliotheken 

Zum Web Scraping wird **BeautifulSoup** verwendet. BeautifulSoup ist eine freie Programmbibliothek für Screen Scraping, mit der sich XML- und HTML-Dokumente parsen lassen.

Zur Speicherung und späteren Abfrage der Informationen wird eine **SQLite-Datenbank** verwendet.  SQLite ist eine C-Bibliothek, die eine leichtgewichtige plattenbasierte Datenbank bereitstellt, die keinen separaten Serverprozess erfordert und den Zugriff auf die Datenbank mit einer nicht standardisierten Variante der SQL-Abfragesprache ermöglicht. Sie eignet sich be manchen Anwendungen zur internen Datenspeicherung. Es ist auch möglich, einen Prototyp einer Anwendung mit SQLite zu erstellen und den Code dann auf eine größere Datenbank wie PostgreSQL oder Oracle zu portieren.


In [27]:
# import of necessary libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import sqlite3

### 1.2. Beschaffung von Namen der HHN Professoren 

Zunächst muss eine Liste mit möglichst vielen HHN Professoren erstellt werden. Diese Liste wird später durchiteriert, um für jeden Professor die HTML-Seite zu parsen und die benötigten Informationen extrahieren zu können. Hier werden Namen von Professoren von einer externen Website gescraped und in eine Liste gespeichert. Diese Liste wird im Ansschluss manuell mit weiteren bekannten HHN Professoren ergänzt.

In [2]:
# implment function to get the professor names from the html with bs4
# output is a list with the names

def get_prof_names():
    url = "https://www.meinprof.de/unis/baden-wuerttemberg/hs-heilbronn"
    page = requests.get(url).text
    doc = BeautifulSoup(page, "html.parser")
    
    names = []
    
    for i in range(284):
        a = doc.find_all(class_ = "bold")
        name = a[i].find("a").contents
        names.append(name)
    
    flat_list = [item for sublist in names for item in sublist]
    
    for i in range(len(flat_list)):
        flat_list[i] = str(flat_list[i])
    
    #remove profs with one name
    profs = []
    for i in range(len(flat_list)):
        if "," in flat_list[i]:
            profs.append(flat_list[i])
    return profs

In [3]:
# calling the function
names = get_prof_names()

In [4]:
# implment function to change the names so that they can be placed at the end of the url
# example: thomas mueller --> thomas.mueller

def transform_name(list):
    prof_name_list = []
    for i in range(len(list)):
        prof_name = list[i]
        prof_name = str(prof_name)
        words = prof_name.split()
        words = words[1], words[0]
        ## joining the words and printing
        prof_name = " ".join(words).strip()
        prof_name = prof_name.replace(",","")
        prof_name = prof_name.replace(" ", ".")
        prof_name_list.append(prof_name)
    return prof_name_list

In [5]:
# calling the function
prof_names = transform_name(names) 

In [6]:
# manually created list with professor names
more_prof_names = ["carsten.lanquillon", "detlef.stern", "jochen.guenther", "helmut.beckmann", "thomas.schaeffer", "sonja.salmen", "ted.azarmi", "katharina.sperl", "rektor", "ulrich.brecht", "ruth.fleuchaus", "raoul.zoellner","christoph.schwerdtfeger"]

In [7]:
# concatenate both lists
all_prof_names=prof_names+more_prof_names
all_prof_names

['Christopher.Adams',
 'Uwe.Ahrens',
 'Tobias.Albrecht',
 'Wolfgang.Albrecht',
 'Martin.Alles',
 'Roland.Alter',
 'Patrick.Balve',
 'Frank.Bantle',
 'Jonathan.Bate',
 'Ralf.Bäuchl',
 'Hans.Bauer',
 'Hans.Bauer',
 'Rainer.Bayer',
 'Helmut.Beckmann',
 'Peter.Bender',
 'Rolf.Bendl',
 'Tomas.Benz',
 'Bleyel.Bernd',
 'Tobias.Bernecker',
 'Thomas.Bezold',
 'Axel.Birk',
 'Lutz.Blecher',
 'Peter.Blessing',
 'Rolf.Blumentritt',
 'Ralf.Bochert',
 'Bernd.Bracht',
 'Bernd.Bracht',
 'Terry.Braemer',
 'David.Braun',
 'Ulrich.Brecht',
 'Birgit.Brigl',
 'Georg.Bucher',
 'Jasmin.Buck',
 'Georg.Bühler',
 'Uwe.Burk',
 'August.Burr',
 'Franzi.Buscholl',
 'Constanze.Clarke',
 'Georg.Clauß',
 'Andreas.Daberkow',
 'Maren-Raphaela.Dechant',
 'Hartmut.Dickhaus',
 'Michael.Dietzel',
 'Ralf.Dillerup',
 'Heiko.Dirks',
 'Juergen.Doneit',
 'Babette.Dorner',
 'Manfred.Dorsch',
 'Michael.Drach',
 'Daniela.Eisele',
 'Wolfgang.Elmendorf',
 'Frank.Engel',
 'Anja.Engelmann',
 'Wolfgang.Ernst',
 'Simon.Fauser',
 'Uwe.Fede

### 1.3. Funktion für Kontaktdaten der Professoren 

Mithilfe von BeautifulSoup wird nun eine Funktion entwickelt, die durch die Liste der Professoren Namen iteriert, den HTML-Code der Profilseiten parst und die benötigten Informationen, wie Name und Email usw., extrahiert und in Listen speichert. Am Ende werden diese Listen in ein Dataframe umgewandelt, dass nun alle Kontaktdaten aller Professoren beinhaltet.


In [8]:
# implement function to extract contact data from the profile sites of the professors

def get_prof_info(list):
    title_list = []
    first_name_list = []
    last_name_list = []
    telephone_list = []
    email_list = []
    office_list = []
    postal_address_list = []
    prof_id_list = []
    
    for prof in list:
        ### get prof_id ###
        prof_id = list.index(prof)+1
        prof_id_list.append(prof_id)
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        
        ### get title, first_name, last_name ### 
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        
        # first_name
        first_name = full_name.split()[-2]
        first_name_list.append(first_name)
        
        # last_name
        last_name = full_name.split()[-1]
        last_name_list.append(last_name)
                
        # title
        title = full_name.replace(first_name, "")
        title = title.replace(last_name, "")
        title_list.append(title)
        
        ### get phone number ###
        telephone = doc.find(class_="mb-2").find(class_="color-teal").string
        telephone_list.append(telephone)
        
        ### get omly the first email ###
        email = doc.find_all(class_="mb-2")[1].find(class_="color-teal").string
        email_list.append(email)
        
        ### get office ###
        if "@" in doc.find_all(class_="mb-2")[2].span.string:
        # go to the next item to get the office
            office = doc.find_all(class_="mb-2")[3].span.string
        else:
            office = doc.find_all(class_="mb-2")[2].span.string
        office_list.append(office)
        
        ### get postal_address ###
        if len(doc.find_all(class_="mb-2")[3].span.string) < 10:
            postal_address = doc.find_all(class_="mb-2")[4].span.string
        else:
            postal_address = doc.find_all(class_="mb-2")[3].span.string
        postal_address_list.append(postal_address)
        
    df = pd.DataFrame({'prof_id': prof_id_list, 'title': title_list, 'first_name': first_name_list, 'last_name': last_name_list, 'telephone': telephone_list, 'email': email_list, 'office': office_list, 'postal_address': postal_address_list})

    return df

In [9]:
# calling the function
prof_info_df = get_prof_info(more_prof_names)

In [10]:
# show the df
prof_info_df

,prof_id,title,first_name,last_name,telephone,email,office,postal_address
0,1,Prof. Dr.-Ing.,Carsten,Lanquillon,+49 7131 504 6942,carsten.lanquillon@hs-heilbronn.de,S.3.45 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
1,2,Prof. Dr.,Detlef,Stern,+49 7131 504 450,detlef.stern@hs-heilbronn.de,S.3.44 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
2,3,Prof. Dr.-Ing.,Jochen,Günther,+49 (0) 7131 504 6945,jochen.guenther@hs-heilbronn.de,S.3.44 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
3,4,Prof. Dr.,Helmut,Beckmann,+49 7131 504 514,helmut.beckmann@hs-heilbronn.de,S.3.47 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
4,5,Dipl.-Inf.,Thomas,Schäffer,+49 7131 504 6699,thomas.schaeffer@hs-heilbronn.de,S.3.46 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
5,6,Prof. Dr.,Sonja-Maria,Salmen,+49 7131 504 477,sonja.salmen@hs-heilbronn.de,S.3.43 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
6,7,Prof. Dr.,Ted,Azarmi,+49 7131 504 6697,ted.azarmi@hs-heilbronn.de,N413 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
7,8,Dr.,Katharina,Sperl,+49 7131 504 519,katharina.sperl@hs-heilbronn.de,N540 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
8,9,Prof. Dr.-Ing.,Oliver,Lenzen,+49 7131 504 201,rektor@hs-heilbronn.de,T.3.26 Bildungscampus,"Max-Planck-Str. 39, 74081 Heilbronn"
9,10,Prof. Dr.,Ulrich,Brecht,+49 7131 504 551,ulrich.brecht@hs-heilbronn.de,T.3.22 Bildungscampus,T.3.22 Bildungscampus


### 1.4. Funktion für Forschungsschwerpunkte

Nach dem gleichen Vorgehen, wie bei den Kontaktdaten, werden nun auch die Forschungschwerpunkte und danach die Studiengänge der Professoren extrahiert. 

In [11]:
# implement function to extract research areas from the profile sites of the professors
def get_research_area(list):
    research_area_list = []
    prof_id_list = []
    last_name_list = []
    

    ### get research areas - Fachgebiete ###
    
    for prof in list:
        ran = True
        prof_id = list.index(prof)+1
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        parent = doc.find_all(class_="text-left")
        
                # last_name
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        last_name = full_name.split()[-1]
        

        
        for i in range(len(parent)):
            if parent[i].string =="Fachgebiete":
                if parent[i+1].find_all("li") != None:
                    child = parent[i+1].find_all("li")
                    for l in range(len(child)):
                        research_area_list.append(child[l].string)
                        prof_id_list.append(prof_id)
                        last_name_list.append(last_name)
            elif parent[i].string =="Forschungsschwerpunkte" or parent[i].string =="Forschungsgebiete":
                if parent[i+1].find_all("li") != None:
                    child = parent[i+1].find_all("li")
                    for l in range(len(child)):
                        research_area_list.append(child[l].string)
                        prof_id_list.append(prof_id)
                        last_name_list.append(last_name)
            else:
                #if doc.select('ul.vertical-list') != None:
                if ran:
                    for i in range(len(doc.select('ul.vertical-list'))):
                        for l in range(len(doc.select('ul.vertical-list')[i])):
                            research_area_list.append(doc.select('ul.vertical-list')[i].find_all("li")[l].string)
                            prof_id_list.append(prof_id)
                            last_name_list.append(last_name)
                            ran = False


    df = pd.DataFrame({'prof_id': prof_id_list,"Name":last_name_list, 'research_area': research_area_list})           
    #print(len(prof_id_list))
    #print(len(research_area_list))
    
    return df

In [12]:
# calling the function
prof_research_area_df = get_research_area(more_prof_names)

In [13]:
# show the df
pd.set_option("display.max_rows", None, "display.max_columns", None)
prof_research_area_df

,prof_id,Name,research_area
0,1,Lanquillon,Business Intelligence
1,1,Lanquillon,Data Science
2,1,Lanquillon,Machine Learning
3,1,Lanquillon,Künstliche Intelligenz
4,1,Lanquillon,Big Data
5,1,Lanquillon,Business Intelligence
6,1,Lanquillon,Big Data Analytics
7,1,Lanquillon,Data Science
8,1,Lanquillon,Data Mining
9,1,Lanquillon,Text Mining


### 1.4. Funktion für Studiengänge

In [14]:
# implement function to extract studies from the profile sites of the professors
def get_study(list):
    study_list = []
    prof_id_list = []
    last_name_list = []
    

    ### get research areas - Fachgebiete ###
    
    for prof in list:
        prof_id = list.index(prof)+1
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        parent = doc.find_all(class_="text-left")
        
                # last_name
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        last_name = full_name.split()[-1]
        
        for i in range(len(doc.find_all(class_ = "color-teal"))):
            study = doc.find_all(class_ = "color-teal")[i].string
            if "+" not in study and  "@" not in study:
                study_list.append(study)
                prof_id_list.append(prof_id)
                last_name_list.append(last_name)
        
    df = pd.DataFrame({'prof_id': prof_id_list,"Name":last_name_list, 'study': study_list})           
    #print(len(prof_id_list))
    #print(len(research_area_list))
    
    return df

In [25]:
# calling the function
prof_study_df = get_study(more_prof_names)

In [27]:
# show the df
prof_study_df

,prof_id,Name,study
0,1,Lanquillon,Wirtschaftsinformatik
1,2,Stern,Wirtschaftsinformatik
2,2,Stern,Wirtschaftsinformatik - Informationsmanagement...
3,2,Stern,Wirtschaftsinformatik - Digitale Transformatio...
4,2,Stern,Fakultät Wirtschaft und Verkehr
5,3,Günther,Wirtschaftsinformatik
6,4,Beckmann,Wirtschaftsinformatik - Informationsmanagement...
7,4,Beckmann,Wirtschaftsinformatik
8,5,Schäffer,Wirtschaftsinformatik
9,6,Salmen,Wirtschaftsinformatik


### 1.5. Dataframes in sqlite Datenbank schreiben 

Diese drei DataFrames werden nun in die SQLite Datenbank gespeichert. Dabei wird jedes df in eine eigene Tabelle gespeichert um so die dritte Normalfunktion erfüllen zu können. Die Datenbank hat den Namen "PROF_INFO_DB".

**Tabellen der Datenbank PROF_INFO_DB:**<br>
PROF_INFO_TABLE --> Kontaktdaten der Professoren <br>
PROF_RESEARCH_AREA_TABLE --> Forschungsschwerpunkte der Professoren <br>
PROF_STUDY_TABLE --> Studiengänge der Professoren


In [16]:
# establish connection to DB, reate the tables and write the dataframes into the tables
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
prof_info_df.to_sql("PROF_INFO_TABLE", conn, index=False)
prof_research_area_df.to_sql("PROF_RESEARCH_AREA_TABLE", conn, index=False)
prof_study_df.to_sql("PROF_STUDY_TABLE", conn, index=False)
conn.close()

NameError: name 'prof_info_df' is not defined

#### SQL Queries testen

In [21]:
conn = sqlite3.connect("../PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select research_area FROM PROF_RESEARCH_AREA_TABLE")
conn.commit()
rows = cur.fetchall()
conn.close()

#rows

### 1.6. Herausforderungen 

* Viele der extrahierten Professoren scheinen nicht mehr an der HHN tätig zu sein, da viele keine Profilseite auf der HHN Website haben. Demnach ist es notwendig auf anderen Websiten nach weiteren HHN Professoren zu suchen.
* Der Aufabu der KOntaktdaten ist auf den Profilseiten der Professoren fast immer gleich. Jedoch varriert die Darstellung der Forschungsschwerpunkte bei manchen Professoren stark vom "Standard". Das heißt die gesuchten Informationen sind bei manchen Professoren nicht in der gleichen HTML-Klasse zu finden wie bei den meisten. 
* Manche Professoren haben ihre Profilseite nicht gepflegt, weshalb hier manche Inforamtionen fehlen.